# Ex9

Uma oficina mecânica deseja alocar o tempo ocioso disponível em suas máquinas para a produção de 3 produtos. A tabela abaixo apresenta as informações sobre as necessidades de horas de máquina para produzir uma unidade de cada produto, assim como a disponibilidade das máquinas, o lucro dos produtos e a demanda máxima existente no mercado. O objetivo é determinar o esquema semanal de produção que maximize o lucro.

## Dados do Problema

| Tipo de máquina         | Produto A | Produto B | Produto C | Tempo disponível (horas por semana) |
|--------------------------|-----------|-----------|-----------|-------------------------------------|
| Torno                   | 5         | 3         | 5         | 400                                 |
| Fresa                   | 8         | 4         | 0         | 500                                 |
| Furadeira               | 2         | 5         | 3         | 300                                 |
| **Lucro por unidade (R$)** | 20        | 15        | 18        |                                     |
| **Demanda mínima semanal** | 40        | 50        | 20        |                                     |




### Modelo

#### Índices:
- $( i )$: Tipos de produtos, onde $( i \in \{ \text{A}, \text{B}, \text{C} \} )$.
- $( j )$: Setores de produção, onde $( j \in \{ \text{torno}, \text{Fresa}, \text{Furadeira}\} )$.

#### Parâmetros:
- $( t_{ij} )$: Tempo de operação (em horas) para produzir o produto $( i )$ no setor $( j )$.
- $( T_j )$: Tempo disponível no operação $( j )$.
- $( D_i )$: Demanda mínima do produto $( i )$.
- $( L_i )$: Lucro do produto $( i )$.

#### Variáveis de Decisão:
- $( x_i )$: Quantidade do produto  $( i )$ a serem fabricados.


#### Função Objetivo:
Maximizar lucro:

$$
Z = \sum_{i} L_i \cdot x_i
$$

#### Restrições:

1. **Tempo disponível :**

$$
\sum_{i} t_{ij} \cdot x_i \leq T_j , \quad \forall j
$$

2. **Demanda mínima:**

$$
 x_i => D_i , \quad \forall i

$$

3. **Não negatividade:**

$$
x_i \geq 0, \quad \forall i
$$


In [11]:
import pyomo.environ as pyo

In [12]:
# Criação do modelo
model = pyo.ConcreteModel()

In [13]:

# Conjuntos
model.P = pyo.Set(initialize=['A', 'B', 'C'])  
model.S = pyo.Set(initialize=['Torno', 'Fresa', 'Furadeira'])  

In [14]:
# ------------------------------
# Parâmetros
# ------------------------------
C = {'A': 20, 'B': 15, 'C': 18}
model.C = pyo.Param(model.P,initialize=C)

D = {'A': 40, 'B': 50, 'C': 20}  # Demanda de cada motor
model.D = pyo.Param(model.P,initialize=D)

tempo_operacao = {
    ('A', 'Torno'): 5, ('A', 'Fresa'): 8, ('A', 'Furadeira'): 2,
    ('B', 'Torno'): 3, ('B', 'Fresa'): 4, ('B', 'Furadeira'): 5,
    ('C', 'Torno'): 5, ('C', 'Fresa'): 0, ('C', 'Furadeira'): 3
}
model.tempo_operacao = pyo.Param(model.P,model.S, initialize=tempo_operacao)

tempo_disponiveis = {'Torno': 400, 'Fresa': 500, 'Furadeira': 300}
model.tempo_disponiveis = pyo.Param(model.S, initialize=tempo_disponiveis)



In [15]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals) 

In [16]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.C[j] * model.x[j] for j in model.P)
model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [17]:
# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de capacidade por operação
def restricao_capacidade(model, i):
    return sum(model.tempo_operacao[j,i] * model.x[j] for j in model.P) <= model.tempo_disponiveis[i]
model.capacidade = pyo.Constraint(model.S, rule=restricao_capacidade)

# 2. Demanda
def restricao_demanda(model, i):
    return model.x[i] <= model.D[i]
model.restricao_demanda = pyo.Constraint(model.P, rule=restricao_demanda)


In [18]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex9.lp", io_options={"symbolic_solver_labels": True})

('ex9.lp', 2643978582000)

In [19]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 6 rows; 3 cols; 11 nonzeros
Coefficient ranges:
  Matrix [1e+00, 8e+00]
  Cost   [2e+01, 2e+01]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 5e+02]
Presolving model
3 rows, 3 cols, 8 nonzeros  0s
3 rows, 3 cols, 8 nonzeros  0s
Presolve : Reductions: rows 3(-3); columns 3(-0); elements 8(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          1    -1.6400000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 1
Objective value     :  1.6400000000e+03
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 1.640,00


In [20]:
for i in model.P:    
    val = pyo.value(model.x[i])
    
    print(f"Quantidade do produto {i} produção própria : {val:.2f}")


Quantidade do produto A produção própria : 40.00
Quantidade do produto B produção própria : 32.00
Quantidade do produto C produção própria : 20.00
